In [ ]:
!pip install -q TTS

In [ ]:
import os, json
from TTS.api import TTS
from tqdm import tqdm
from google.colab import files
uploaded = files.upload()

with open("samples.json", "r", encoding="utf-8") as f:
    all_samples = json.load(f)

half = len(all_samples) // 2
samples_split = {
    "tts_models/it/mai_female/glow-tts": {
        "samples": all_samples[:half],
        "gender": "female"
    },
    "tts_models/it/mai_male/glow-tts": {
        "samples": all_samples[half:],
        "gender": "male"
    }
}

base_output_dir = "coqui_output"
os.makedirs(base_output_dir, exist_ok=True)

for model_name, data in samples_split.items():
    samples = data["samples"]
    gender = data["gender"]
    subfolder = os.path.join(base_output_dir, gender)
    os.makedirs(subfolder, exist_ok=True)

    tts = TTS(model_name=model_name)

    for i, sample in tqdm(enumerate(samples), total=len(samples), desc=f"{gender}"):
        text = sample["text"]
        output_filename = f"sample_{i}_{gender}.wav"
        output_path = os.path.join(subfolder, output_filename)

        tts.tts_to_file(text=text, file_path=output_path)

        sample_copy = sample.copy()
        sample_copy["coqui_audio_path"] = os.path.join(gender, output_filename)
        sample_copy["coqui_model"] = model_name
        sample_copy["speaker_gender"] = gender
        samples[i] = sample_copy

    with open(os.path.join(subfolder, "samples_coqui.json"), "w", encoding="utf-8") as f:
        json.dump(samples, f, indent=2, ensure_ascii=False)

# Merge all samples into one global file
all_final_samples = []

for gender in ["female", "male"]:
    with open(os.path.join(base_output_dir, gender, "samples_coqui.json"), "r", encoding="utf-8") as f:
        all_final_samples.extend(json.load(f))

# Save global metadata
with open(os.path.join(base_output_dir, "all_samples_coqui.json"), "w", encoding="utf-8") as f:
    json.dump(all_final_samples, f, indent=2, ensure_ascii=False)

print("Global metadata saved at:", os.path.join(base_output_dir, "all_samples_coqui.json"))

# Create a single zip file containing both folders
!zip -r coqui_output.zip coqui_output
files.download("coqui_output.zip")